In [1]:
import pandas as pd
import jaconv
from bs4 import BeautifulSoup 
from urllib import request
from urllib.error import HTTPError
import matplotlib.pyplot as plt
import numpy as np
import pickle
import datetime

# Change fontype of pyplot
import matplotlib.font_manager
#print([f.name for f in matplotlib.font_manager.fontManager.ttflist])
plt.rcParams['font.family'] = 'MS Gothic'

# If you want to downlod the most recent data then you should not skip this.
skip = 0

Define urls

In [2]:
# url headers
url_head = "https://www5.cao.go.jp/keizai3/"
url_new = "https://www5.cao.go.jp/keizai3/watcher_index.html"
url_old = "https://www5.cao.go.jp/keizai3/kako_watcher.html"
# 
kigou2value = {"◎":5,"○":4,"□":3,"▲":2,"×":1}

Define functions

In [3]:
def get_all_url(target):
    '''
    Description:webページ内の全urlを取得する関数
    入力：各調査が一覧になったページ
    出力：各調査のcsvがあるurl一覧
    '''
    data = request.urlopen(target).read()
    soup = BeautifulSoup(data, 'html.parser')
    tmp_links = soup.find_all("a")
    link = []
    for a in tmp_links:
        try:
            if "/menu.html" in a.attrs['href']:
                link.append(a.attrs['href'])
        except:
            pass
    return link


def extract_data(df_in):
    '''
    Description:各調査のcsvがあるurlからデータフレームを直接作る関数
    入力：各調査のcsvに直接アクセスできるurl
    出力：データフレーム
    '''  
    han_numbers = [str(_) for _ in range(10)]
    kigou_list = ["◎","○","□","▲","×"]
    region_list = ["北海道","東北","北関東","南関東","甲信越",
                   "東海","北陸","近畿","中国","四国","九州","沖縄"]
    
    place = 6
    sakiyuki = 0
    for l in range(len(df_in)):
        if ('景気の現状判断' in str(df_in.iloc[l,:])):
            place = l
            sakiyuki = 0
            break
            
        if ("景気の先行き判断" in str(df_in.iloc[l,:])):
            place = l
            sakiyuki = 1
            break
    
    col_names = df_in.iloc[place,:]
    use_col_names = dict()
    for l in range(len(col_names)):
        use_col_names.update({col_names[l]:l})
        
    out_data = []
    
    # Used to determine which regions
    region = ""

    # Loop through all records
    for i in range(len(df_in)):
        line = df_in.iloc[i,0]

        # Update line headers
        if line == line:
            new_region = 0
            for r in region_list:
                if r in line:
                    new_region = 1
                    break
                   
            if new_region == 1:
                region = r
        

        line = df_in.iloc[i,2]
        if line == line:
            for kigou in kigou_list:
                if kigou in line:
                    if sakiyuki == 0:
                    
                        if "判断の理由" in use_col_names:

                            out_data.append([region,
                                             df_in.iloc[i,use_col_names["景気の現状判断"]],
                                             df_in.iloc[i,use_col_names["業種・職種"]],
                                             df_in.iloc[i,use_col_names["判断の理由"]],
                                             df_in.iloc[i,use_col_names["追加説明及び具体的状況の説明"]]])
                        else:
                            out_data.append([region,
                                             df_in.iloc[i,use_col_names["景気の現状判断"]],
                                             df_in.iloc[i,use_col_names["業種・職種"]],
                                             "",
                                             df_in.iloc[i,use_col_names["追加説明及び具体的状況の説明"]]])
                            
                    else:
                        if "判断の理由" in use_col_names:

                            out_data.append([region,
                                             df_in.iloc[i,use_col_names["景気の先行き判断"]],
                                             df_in.iloc[i,use_col_names["業種・職種"]],
                                             df_in.iloc[i,use_col_names["判断の理由"]],
                                             df_in.iloc[i,use_col_names["景気の先行きに対する判断理由"]]])
                        else:
                            out_data.append([region,df_in.iloc[i,use_col_names["景気の先行き判断"]],
                                             df_in.iloc[i,use_col_names["業種・職種"]],
                                             "",
                                             df_in.iloc[i,use_col_names["景気の先行きに対する判断理由"]]])     
                        
    df_out = pd.DataFrame(out_data)
    df_out.columns = ["region","symbol","industry","topic","text"]
    
    return(df_out)

In [4]:
# retrieve all urls
all_urls = get_all_url(url_new)
all_urls.extend(get_all_url(url_old))
all_urls.sort()

In [8]:
%%time
if skip != 1:
    error_urls, list_dfs = [],[]
    #for i in range(257,258):
    for i in range(len(all_urls)):
        print(i)
        date = all_urls[i][0:4] +"_" +  all_urls[i][5:7] + "_" +  all_urls[i][7:9] 

        # 現状
        file_url = url_head + all_urls[i].strip("menu.html") + "watcher4.csv"
        
        try:
            df_tmp = pd.read_csv(file_url,encoding="cp932")
            df_out = extract_data(df_tmp)
            df_out["date"] = date
            df_out["type"] = "now"
            list_dfs.append(df_out)

        except:
            error_urls.append(file_url)
            
        # 先行き
        file_url = url_head + all_urls[i].strip("menu.html") + "watcher5.csv"

        try:
            df_tmp = pd.read_csv(file_url,encoding="cp932")
            df_out = extract_data(df_tmp)
            df_out["date"] = date
            df_out["type"] = "future"
            list_dfs.append(df_out)

        except:
            #print(file_url)
            error_urls.append(file_url)

        # 甲信越　現状
        file_url = url_head + all_urls[i].strip("menu.html") + "watcher6.csv"
        try:
            df_tmp = pd.read_csv(file_url,encoding="cp932")
            df_out = extract_data(df_tmp)
            df_out["date"] = date
            df_out["type"] = "now_koushin"
            list_dfs.append(df_out)

        except:
            error_urls.append(file_url)

        # 甲信越　先行き
        file_url = url_head + all_urls[i].strip("menu.html") + "watcher7.csv"
        try:
            df_tmp = pd.read_csv(file_url,encoding="cp932")
            df_out = extract_data(df_tmp)
            df_out["date"] = date
            df_out["type"] = "future_koushin"
            list_dfs.append(df_out)

        except:
            error_urls.append(file_url)

0
1


/var/folders/xn/9jbxxjcj63b3xbh21qlt_pcr0000gn/T/ipykernel_83080/935186160.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})
/var/folders/xn/9jbxxjcj63b3xbh21qlt_pcr0000gn/T/ipykernel_83080/935186160.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})


2
3
4
5


/var/folders/xn/9jbxxjcj63b3xbh21qlt_pcr0000gn/T/ipykernel_83080/935186160.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})
/var/folders/xn/9jbxxjcj63b3xbh21qlt_pcr0000gn/T/ipykernel_83080/935186160.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})


6
7
8
9


...

175


/var/folders/xn/9jbxxjcj63b3xbh21qlt_pcr0000gn/T/ipykernel_83080/935186160.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})
/var/folders/xn/9jbxxjcj63b3xbh21qlt_pcr0000gn/T/ipykernel_83080/935186160.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})
/var/folders/xn/9jbxxjcj63b3xbh21qlt_pcr0000gn/T/ipykernel_83080/935186160.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by positi

176


/var/folders/xn/9jbxxjcj63b3xbh21qlt_pcr0000gn/T/ipykernel_83080/935186160.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})
/var/folders/xn/9jbxxjcj63b3xbh21qlt_pcr0000gn/T/ipykernel_83080/935186160.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  use_col_names.update({col_names[l]:l})
/var/folders/xn/9jbxxjcj63b3xbh21qlt_pcr0000gn/T/ipykernel_83080/935186160.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by positi

177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
CPU times: user 1min 42s, sys: 4.88 s, total: 1min 47s
Wall time: 5min 10s


In [ ]:
# Concatenate all together

if skip != 1:
    # concatenate
    df = pd.concat(list_dfs)
    # change symbols to values
    df["label"] = df["symbol"].map(kigou2value)
    # count type
    df["type"].value_counts()

# Save
if skip != 1:
    #save
    df.to_csv("./keiki.csv", index = False)